# BERTimbau

O modelo `neuralmind/bert-base-portuguese-cased` é uma versão do BERT (Bidirectional Encoder Representations from Transformers) pré-treinado para a língua portuguesa. Aqui estão algumas informações sobre esse modelo específico:

1. **Pré-treinamento:**
   - Este modelo foi treinado previamente em uma tarefa de máscara de linguagem (Masked Language Model, MLM) e em tarefas adicionais como "next sentence prediction" (previsão da próxima sentença). Durante o treinamento prévio, o modelo aprendeu representações contextualizadas de palavras e frases em português.

2. **Dominio de Aplicação:**
   - O modelo pode ser aplicado a uma variedade de tarefas relacionadas ao processamento de linguagem natural (PLN) para a língua portuguesa. Isso inclui, mas não está limitado a, tarefas de classificação de texto, geração de texto, análise de sentimento, identificação de entidades, tradução automática e muito mais.

3. **Tarefas Compatíveis:**
   - O BERT é um modelo transformer pré-treinado, e a versão específica para o português, `neuralmind/bert-base-portuguese-cased`, pode ser usada em uma ampla variedade de tarefas de PLN. Algumas tarefas comuns incluem:
     - **Classificação de Texto:** Classificação binária (duas classes) ou multiclasse (mais de duas classes).
     - **Análise de Sentimento:** Classificação do sentimento em positivo, negativo ou neutro.
     - **NER (Named Entity Recognition):** Identificação e classificação de entidades nomeadas no texto.
     - **Tradução Automática:** Geração de traduções em português para outros idiomas e vice-versa.
     - **Pergunta e Resposta:** Responder a perguntas com base em um determinado contexto.

4. **Tokens e Casing:**
   - O modelo `neuralmind/bert-base-portuguese-cased` é "cased", o que significa que ele mantém distinção entre maiúsculas e minúsculas. Ele também utiliza tokenização WordPiece, o que significa que o texto é dividido em subpalavras.

Para usar o modelo em uma tarefa específica, você normalmente ajustaria ou "afinaria" o modelo em uma tarefa de ajuste fino (fine-tuning) com dados específicos da sua aplicação. Por exemplo, você poderia ajustar o modelo para uma tarefa de classificação binária ou multiclasse usando seus próprios dados rotulados.

In [40]:
import torch
from transformers import AutoModel, AutoTokenizer, BertForSequenceClassification
from torch.nn import Softmax
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
from torch.optim import AdamW

In [41]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

Esse trecho de código está relacionado ao uso da biblioteca Transformers da Hugging Face para trabalhar com modelos BERT (Bidirectional Encoder Representations from Transformers) em tarefas de classificação de sequência.

Aqui está uma explicação detalhada:

1. **`model_name = 'neuralmind/bert-base-portuguese-cased'`:**
   - Define o nome do modelo pré-treinado que você deseja usar. Neste caso, 'neuralmind/bert-base-portuguese-cased' refere-se a uma versão do modelo BERT pré-treinado em português, fornecido pela NeuralMind.

2. **`tokenizer = BertTokenizer.from_pretrained(model_name)`:**
   - Cria um tokenizador específico para o modelo BERT escolhido (`'neuralmind/bert-base-portuguese-cased'`). O tokenizador é responsável por converter o texto em tokens, que são as unidades de entrada que o modelo BERT entende. Ele também executa tarefas como dividir palavras em subpalavras e adicionar tokens especiais.

3. **`model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)`:**
   - Carrega o modelo BERT pré-treinado para uma tarefa específica, que é a classificação de sequências. `BertForSequenceClassification` é uma classe que é uma extensão do modelo BERT básico, projetada para tarefas de classificação. Neste caso, o parâmetro `num_labels=2` indica que você está lidando com uma tarefa de classificação binária, onde há duas classes possíveis.

Resumindo, essas linhas de código configuram um ambiente para usar um modelo BERT pré-treinado em português, com um tokenizador adequado e um modelo de classificação de sequência adaptado para sua tarefa específica. Isso é útil, por exemplo, para treinar o modelo em um conjunto de dados específico de classificação de texto em português.

21726 registros

In [42]:
#data = pd.read_csv('hate_speech_ptbr.csv')  # lendo o dastaset CSV
data = pd.read_csv('Base_limpissima.csv')

In [43]:
data.head()

,labels,tweets
0,not_hatetag,geral foi no jogo do foz
1,not_hatetag,pois é não é gente é isso ai pessoal
2,not_hatetag,ontem eu estava tão mal que escrevi tudo o que...
3,not_hatetag,eu devo ter um coração de pedra não sou de me ...
4,not_hatetag,vivo br pois é estou tentando entender


In [44]:
data.columns

Index(['labels', 'tweets'], dtype='object')

In [45]:
labels_agrupados = data.groupby('labels')
print(labels_agrupados.size())

labels
hate            2594
not_hatetag    19250
dtype: int64


In [46]:
# Carregar os dados
#df = pd.read_csv('Base_limpissima.csv')  # Substitua 'seu_dataset.csv' pelo nome do seu arquivo CSV
tweets = data['tweets'].values
labels = data['labels'].values

In [47]:
# Carregar o modelo BERT pré-treinado e o tokenizador
model_name = 'neuralmind/bert-base-portuguese-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Esse trecho de código está relacionado ao processo de tokenização dos dados de texto usando o tokenizador BERT para prepará-los para alimentar o modelo BERT. Vamos explicar cada parte:

1. **`tokenizer(list(tweets_train), padding=True, truncation=True, return_tensors='pt', max_length=128)`:**
   - `tokenizer`: Chama o tokenizador BERT previamente carregado.

   - `list(tweets_train)`: Converte a lista de textos (`tweets_train`) em uma lista Python, que é aceita como entrada pelo tokenizador.

   - `padding=True`: Solicita ao tokenizador para adicionar tokens de preenchimento para garantir que todas as sequências tenham o mesmo comprimento.

   - `truncation=True`: Solicita ao tokenizador para truncar ou cortar sequências que excedem o comprimento máximo permitido (`max_length`).

   - `return_tensors='pt'`: Solicita ao tokenizador para retornar os tokens como tensores PyTorch (`'pt'`).

   - `max_length=128`: Limita o comprimento máximo das sequências tokenizadas a 128 tokens. Sequências mais longas serão truncadas.

   O resultado dessa chamada ao tokenizador (`tokens_train`) será um dicionário contendo os tokens resultantes, as máscaras de atenção e outros detalhes necessários para alimentar o modelo BERT durante o treinamento ou inferência.

2. **`tokenizer(list(tweets_test), padding=True, truncation=True, return_tensors='pt', max_length=128)`:**
   - Similar ao anterior, mas aplicado ao conjunto de teste (`tweets_test`). Isso garante que o conjunto de teste seja tokenizado da mesma maneira que o conjunto de treinamento.

Essencialmente, essa etapa de tokenização transforma os textos em sequências de tokens que o modelo BERT pode entender. As opções de preenchimento, truncamento e limitação de comprimento garantem que as sequências estejam padronizadas antes de serem alimentadas ao modelo, o que é necessário para o funcionamento adequado do modelo BERT.

In [ ]:
# Converter labels para números
#label_mapping = {"hate": 0, "not_hate": 1}  # para base haste_speech_ptbr.csv
label_mapping = {"hate": 0, "not_hatetag": 1} # para a base Base_limpissima.csv
labels = [label_mapping[label] for label in labels]

# Dividir o conjunto de dados em treinamento e teste
tweets_train, tweets_test, labels_train, labels_test = train_test_split(tweets, labels, test_size=0.2, random_state=42)

# Tokenizar os dados
tokens_train = tokenizer(list(tweets_train), padding=True, truncation=True, return_tensors='pt', max_length=64)
tokens_test = tokenizer(list(tweets_test), padding=True, truncation=True, return_tensors='pt', max_length=64)

Esse trecho de código está criando conjuntos de dados personalizados usando a biblioteca PyTorch para treinamento e teste. Vamos explicar cada parte:

1. **`class CustomDataset(Dataset):`**
   - Define uma nova classe chamada `CustomDataset` que herda da classe `Dataset` do PyTorch. Isso é feito para criar conjuntos de dados personalizados que podem ser utilizados pelo DataLoader do PyTorch durante o treinamento e teste.

2. **`def __init__(self, tokens, labels):`**
   - O método `__init__` é o construtor da classe e é chamado quando você cria uma instância da classe. Ele recebe dois argumentos: `tokens` e `labels`.

   - `self.tokens = tokens`: Armazena os tokens (resultados da tokenização) no atributo `self.tokens`.

   - `self.labels = torch.tensor(labels, dtype=torch.long)`: Armazena as etiquetas convertidas em tensores PyTorch no atributo `self.labels`. O `dtype=torch.long` é usado para garantir que as etiquetas sejam representadas como números inteiros longos.

3. **`def __len__(self):`**
   - Define o método `__len__` que retorna o tamanho do conjunto de dados, que é o número de amostras. Neste caso, é o comprimento do atributo `self.tokens['input_ids']`.

4. **`def __getitem__(self, idx):`**
   - Define o método `__getitem__` que é chamado quando você acessa um item do conjunto de dados usando a notação de índice (`dataset[idx]`). Recebe um índice `idx` como argumento.

   - Retorna um dicionário contendo os tokens para a amostra de índice `idx` (`{key: tensor[idx] for key, tensor in self.tokens.items()}`) e a etiqueta correspondente (`self.labels[idx]`).

5. **`train_dataset = CustomDataset(tokens_train, labels_train)` e `test_dataset = CustomDataset(tokens_test, labels_test)`**
   - Cria instâncias do conjunto de dados personalizado (`CustomDataset`) para treinamento (`train_dataset`) e teste (`test_dataset`) usando os tokens e etiquetas correspondentes. Isso permite que você utilize esses conjuntos de dados ao treinar e avaliar modelos no PyTorch.

Esse código é útil para encapsular seus dados em um formato que seja compatível com as funcionalidades do PyTorch, especialmente quando você está trabalhando com conjuntos de dados personalizados ou estruturas específicas para tarefas complexas, como o treinamento de modelos de linguagem usando BERT.

In [49]:
# Criar conjuntos de dados do PyTorch
class CustomDataset(Dataset):
    def __init__(self, tokens, labels):
        self.tokens = tokens
        self.labels = torch.tensor(labels, dtype=torch.long)  # Alterado para torch.long

    def __len__(self):
        return len(self.tokens['input_ids'])

    def __getitem__(self, idx):
        return {key: tensor[idx] for key, tensor in self.tokens.items()}, self.labels[idx]

train_dataset = CustomDataset(tokens_train, labels_train)
test_dataset = CustomDataset(tokens_test, labels_test)

# Inicialização dos parametros do treinamento

Esse trecho de código define parâmetros importantes para o treinamento de um modelo. Aqui está a explicação de cada linha:

1. **`batch_size = 4`**:
   - Define o tamanho do lote (batch size). O tamanho do lote representa o número de exemplos de treinamento que são processados em uma única iteração (passo) de treinamento. Um tamanho de lote menor, como 4 neste caso, significa que o modelo será atualizado mais frequentemente, mas cada atualização será baseada em menos exemplos. O tamanho do lote é um hiperparâmetro que pode afetar o desempenho e a eficiência do treinamento.

2. **`learning_rate = 2e-5`**:
   - Define a taxa de aprendizado (learning rate). A taxa de aprendizado determina o tamanho dos passos que o otimizador dá ao ajustar os pesos do modelo durante o treinamento. Uma taxa de aprendizado pequena significa que os ajustes são feitos em pequenos incrementos, enquanto uma taxa de aprendizado grande pode resultar em ajustes mais significativos. A escolha da taxa de aprendizado é crítica para o sucesso do treinamento e geralmente requer ajustes e experimentação.

3. **`epochs = 1`**:
   - Define o número de épocas. Uma época é uma passagem completa por todo o conjunto de treinamento. Um valor de 1 para o número de épocas significa que o modelo verá todo o conjunto de treinamento uma vez durante o treinamento. O número de épocas é um hiperparâmetro que determina quantas vezes o modelo irá treinar em todo o conjunto de dados. Pode ser ajustado com base na convergência do modelo.

Esses parâmetros são críticos para o treinamento bem-sucedido de modelos de aprendizado de máquina. Experimentar diferentes valores para esses parâmetros pode ser necessário para encontrar a combinação que resulta no melhor desempenho para uma tarefa específica. O processo de ajuste desses parâmetros é chamado de ajuste de hiperparâmetros e geralmente envolve tentativa e erro para encontrar a combinação mais eficaz para o seu problema.

In [50]:
# Definir parâmetros de treinamento
batch_size = 8  # tamanho do lote, pode ser 6, 7, etc...
learning_rate = 3e-5
epochs = 3  # número de épocas

Esse trecho de código está relacionado à preparação de DataLoader para facilitar o treinamento e a avaliação do modelo no PyTorch. Vamos explicar cada linha:

1. **`train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)`**:
   - Cria um DataLoader para o conjunto de treinamento (`train_dataset`). O DataLoader é uma classe do PyTorch que permite iterar eficientemente sobre os dados durante o treinamento do modelo.

   - `train_dataset`: É o conjunto de dados personalizado que foi criado anteriormente.

   - `batch_size`: Especifica o tamanho do lote, ou seja, o número de amostras que serão processadas de uma vez durante cada iteração de treinamento.

   - `shuffle=True`: Embaralha os dados no DataLoader antes de cada época de treinamento. Embaralhar os dados é importante para garantir que o modelo não aprenda a depender da ordem específica das amostras no conjunto de treinamento.

2. **`test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)`**:
   - Cria um DataLoader para o conjunto de teste (`test_dataset`). A configuração é semelhante à do DataLoader de treinamento, mas `shuffle` é definido como `False` para manter a ordem original dos dados de teste.

Resumidamente, esses DataLoaders são úteis durante o treinamento e avaliação do modelo, pois fornecem iterações eficientes sobre os dados. Durante cada iteração, o DataLoader retorna um lote de dados (um conjunto de entradas e suas respectivas etiquetas) que podem ser usados para calcular gradientes, realizar atualizações nos pesos do modelo e avaliar o desempenho do modelo. Essa abordagem é eficiente, especialmente ao lidar com grandes conjuntos de dados que não podem ser carregados inteiramente na memória.



In [51]:
# Preparar DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Esse trecho de código está relacionado à preparação do otimizador e da função de perda para o treinamento do modelo. Vamos explicar cada linha:

1. **`optimizer = AdamW(model.parameters(), lr=learning_rate)`**:
   - Cria um otimizador do tipo AdamW para ajustar os pesos do modelo durante o treinamento. O otimizador é uma ferramenta que implementa um algoritmo de otimização para ajustar os pesos do modelo com base nos gradientes calculados durante a retropropagação.

   - `model.parameters()`: Fornece os parâmetros do modelo que precisam ser otimizados. Aqui, assume-se que `model` é o modelo definido anteriormente.

   - `lr=learning_rate`: Define a taxa de aprendizado do otimizador. A taxa de aprendizado controla o tamanho dos passos que o otimizador toma ao ajustar os pesos do modelo.

   - `AdamW`: É uma variação do otimizador Adam, que é uma técnica de otimização popular em aprendizado profundo. A versão "W" do AdamW inclui uma correção de peso, sendo especialmente útil com modelos de aprendizado de máquina baseados em transformadores, como o BERT.

2. **`criterion = torch.nn.CrossEntropyLoss()`**:
   - Cria uma instância da função de perda CrossEntropyLoss. A função de perda é uma medida que quantifica o quão bem o modelo está performando em relação à tarefa específica (neste caso, classificação). A CrossEntropyLoss é frequentemente usada em tarefas de classificação com mais de duas classes.

   - No entanto, no seu caso, você está trabalhando com uma tarefa de classificação binária (duas classes), então talvez seja mais apropriado usar `torch.nn.BCEWithLogitsLoss()` se os resultados do seu modelo estiverem no formato de logits (antes da aplicação de uma função de ativação).

Resumindo, esse trecho de código prepara os elementos essenciais para treinar um modelo. O otimizador é responsável por atualizar os pesos do modelo, e a função de perda é usada para calcular o quão bem o modelo está se saindo em comparação com as etiquetas reais durante o treinamento.

Além do AdamW, existem vários outros otimizadores que podem ser utilizados no treinamento de modelos de aprendizado de máquina, incluindo modelos baseados em BERT. Alguns dos otimizadores mais comuns incluem:

1. **SGD (Stochastic Gradient Descent):**
   - O otimizador clássico de gradiente descendente estocástico. Pode ser usado, mas geralmente requer uma boa escolha de taxa de aprendizado.

```python
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
```

2. **Adam (Adaptive Moment Estimation):**
   - Uma versão relacionada ao AdamW, que é amplamente utilizado devido à sua eficácia em muitos cenários.

```python
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
```

3. **Adagrad (Adaptive Gradient Algorithm):**
   - Adapta a taxa de aprendizado individualmente para cada parâmetro com base na história dos gradientes.

```python
optimizer = torch.optim.Adagrad(model.parameters(), lr=learning_rate)
```

4. **Adadelta:**
   - Uma versão modificada do Adagrad que tenta resolver alguns dos problemas relacionados à diminuição da taxa de aprendizado.

```python
optimizer = torch.optim.Adadelta(model.parameters(), lr=learning_rate)
```

5. **RMSprop (Root Mean Square Propagation):**
   - Também é um otimizador adaptativo que ajusta a taxa de aprendizado individualmente para cada parâmetro.

```python
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
```

6. **L-BFGS (Limited-memory Broyden-Fletcher-Goldfarb-Shanno):**
   - Um otimizador quasi-Newton que utiliza uma aproximação do inverso da matriz hessiana.

```python
optimizer = torch.optim.LBFGS(model.parameters(), lr=learning_rate)
```

A escolha do otimizador pode depender da natureza específica do problema, da arquitetura do modelo e da experiência empírica. O AdamW é uma escolha comum para modelos BERT e tem se mostrado eficaz em muitos cenários, mas experimentar diferentes otimizadores pode ser útil para encontrar a melhor configuração para o seu caso específico.

In [52]:
# Preparar otimizador e função de perda

optimizer = AdamW(model.parameters(), lr=learning_rate)
criterion = torch.nn.CrossEntropyLoss()




Este trecho de código está relacionado à configuração do dispositivo de treinamento e à preparação do modelo para o treinamento. Vamos explicar cada linha:

1. **`device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')`**:
   - Verifica se uma GPU (CUDA) está disponível no sistema. Se uma GPU estiver disponível, o dispositivo (`device`) será configurado para 'cuda', caso contrário, será configurado para 'cpu'. Isso permite que o código seja executado em GPU se disponível, caso contrário, usará a CPU.

2. **`model.to(device)`**:
   - Move o modelo para o dispositivo especificado (GPU ou CPU). Isso é necessário para garantir que o modelo e os dados estejam no mesmo dispositivo para o treinamento. Se a GPU estiver disponível, mover o modelo para a GPU pode acelerar significativamente o treinamento, pois as operações podem ser realizadas paralelamente.

3. **`model.train()`**:
   - Coloca o modelo no modo de treinamento. Isso é importante porque alguns modelos, especialmente aqueles que utilizam técnicas de regularização ou camadas específicas para o treinamento, podem se comportar de maneira diferente durante o treinamento e a avaliação. O método `train()` ativa esses comportamentos específicos do treinamento no modelo.

Resumindo, essas linhas de código configuram o dispositivo de treinamento (GPU ou CPU), movem o modelo para esse dispositivo e colocam o modelo no modo de treinamento. Isso é uma prática comum ao treinar modelos em PyTorch para garantir que o treinamento ocorra de maneira eficiente e que o modelo esteja em um estado consistente para aprendizado de parâmetros.

In [ ]:
# Treinar o modelo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.train()

Este trecho de código representa o loop de treinamento principal para o modelo. Vamos explicar cada parte:

1. **`for epoch in range(epochs):`**:
   - Inicia um loop sobre o número de épocas definido anteriormente. Cada época representa uma passagem completa pelos dados de treinamento.

2. **`total_loss = 0`**:
   - Inicializa uma variável `total_loss` que será usada para acumular as perdas durante o treinamento.

3. **`for batch in train_dataloader:`**:
   - Inicia um loop sobre os lotes de dados no DataLoader de treinamento. Para cada lote, os dados são carregados em tensores no dispositivo de treinamento (CPU ou GPU).

4. **`input_ids = batch[0]['input_ids'].to(device)`**, **`attention_mask = batch[0]['attention_mask'].to(device)`**, **`labels = batch[1].to(device)`**:
   - Extrai os tensores de entrada (`input_ids` e `attention_mask`) e as etiquetas (`labels`) do lote. Esses tensores são movidos para o dispositivo de treinamento (CPU ou GPU) usando o método `.to(device)`.

5. **`optimizer.zero_grad()`**:
   - Limpa os gradientes acumulados no otimizador. Isso é necessário antes de calcular os gradientes para um novo lote.

6. **`outputs = model(input_ids, attention_mask=attention_mask, labels=labels)`**:
   - Realiza uma passagem para frente (forward pass) do modelo com os dados do lote e calcula as previsões e a perda associada.

7. **`loss = outputs.loss`**:
   - Extrai a perda do objeto de saída (`outputs`). A perda é uma medida do quão longe as previsões do modelo estão das etiquetas verdadeiras.

8. **`total_loss += loss.item()`**:
   - Acumula a perda no `total_loss`. O método `.item()` é usado para obter o valor escalar da perda a partir de um tensor PyTorch.

9. **`loss.backward()`**:
   - Calcula os gradientes dos parâmetros do modelo em relação à perda. Isso é feito automaticamente pelo PyTorch durante a retropropagação.

10. **`optimizer.step()`**:
    - Atualiza os parâmetros do modelo usando o otimizador, com base nos gradientes calculados durante a retropropagação.

11. **`avg_loss = total_loss / len(train_dataloader)`**:
    - Calcula a média da perda total ao longo de todos os lotes no conjunto de treinamento.

12. **`print(f'Epoch {epoch + 1}/{epochs}, Average Training Loss: {avg_loss:.4f}')`**:
    - Imprime a média da perda de treinamento para a época atual. Isso fornece uma indicação do desempenho do modelo durante o treinamento.

Esse loop de treinamento é fundamental para ajustar os pesos do modelo com base nos dados de treinamento, permitindo que o modelo aprenda a realizar a tarefa específica para a qual está sendo treinado. A perda média é uma métrica comumente usada para avaliar o desempenho do modelo durante o treinamento.

In [54]:
for epoch in range(epochs):
    total_loss = 0
    for batch in train_dataloader:
        input_ids = batch[0]['input_ids'].to(device)
        attention_mask = batch[0]['attention_mask'].to(device)
        labels = batch[1].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

    avg_loss = total_loss / len(train_dataloader)
    print(f'Epoch {epoch + 1}/{epochs}, Average Training Loss: {avg_loss:.4f}')

Epoch 1/3, Average Training Loss: 0.0774
Epoch 2/3, Average Training Loss: 0.0363
Epoch 3/3, Average Training Loss: 0.0222


Este trecho de código avalia o modelo em uma amostra menor do conjunto de teste. Vamos explicar cada parte:

1. **`model.eval()`**:
   - Coloca o modelo no modo de avaliação. Isso é importante porque alguns modelos, especialmente aqueles que utilizam técnicas como dropout ou batch normalization, podem se comportar de maneira diferente durante a avaliação em comparação com o treinamento. O método `eval()` desativa esses comportamentos específicos do treinamento.

2. **`all_preds = []`**:
   - Inicializa uma lista vazia `all_preds` para armazenar todas as previsões do modelo.

3. **`with torch.no_grad():`**:
   - Entra em um contexto onde as operações não geram gradientes. Isso é útil durante a avaliação, pois não precisamos calcular gradientes e economizamos recursos computacionais.

4. **`for batch in test_dataloader:`**:
   - Inicia um loop sobre os lotes de dados no DataLoader de teste.

5. **`input_ids = batch[0]['input_ids'].to(device)`**, **`attention_mask = batch[0]['attention_mask'].to(device)`**, **`labels = batch[1].to(device)`**:
   - Extrai os tensores de entrada (`input_ids` e `attention_mask`) e as etiquetas (`labels`) do lote de teste. Os tensores são movidos para o dispositivo de avaliação (CPU ou GPU).

6. **`outputs = model(input_ids, attention_mask=attention_mask)`**:
   - Realiza uma passagem para frente (forward pass) do modelo com os dados de teste, mas sem fornecer as etiquetas. Calcula as previsões do modelo (`logits`).

7. **`preds = torch.argmax(logits, dim=1).cpu().numpy()`**:
   - Calcula as previsões finais do modelo. O método `torch.argmax` retorna os índices dos valores máximos ao longo de uma dimensão, neste caso, ao longo da dimensão 1 (que representa as classes). `cpu().numpy()` é usado para transferir os resultados para a CPU e convertê-los em um array NumPy.

8. **`all_preds.extend(preds)`**:
   - Adiciona as previsões do lote à lista `all_preds`.

9. **`accuracy = accuracy_score(labels_test[:len(all_preds)], all_preds)`**:
   - Calcula a acurácia comparando as previsões do modelo com as etiquetas reais no conjunto de teste.

10. **`print(f'Accuracy on Test Set: {accuracy:.4f}')`**:
    - Imprime a acurácia no conjunto de teste.

11. **`print(classification_report(labels_test[:len(all_preds)], all_preds))`**:
    - Imprime um relatório de classificação que inclui várias métricas, como precisão, recall e F1-score, para avaliar o desempenho do modelo nas diferentes classes.

Este trecho de código é útil para obter uma visão rápida do desempenho do modelo em uma amostra menor do conjunto de teste. Note que, para avaliações mais detalhadas, é recomendável usar o conjunto de teste completo. Além disso, dependendo dos requisitos específicos, você pode ajustar a amostra de teste usada para avaliação.

In [55]:
# Avaliar o modelo em uma amostra menor do conjunto de teste
model.eval()
all_preds = []

with torch.no_grad():
    for batch in test_dataloader:
        input_ids = batch[0]['input_ids'].to(device)
        attention_mask = batch[0]['attention_mask'].to(device)
        labels = batch[1].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1).cpu().numpy()
        all_preds.extend(preds)

# Calcular a acurácia na amostra menor do conjunto de teste
accuracy = accuracy_score(labels_test[:len(all_preds)], all_preds)
print(f'Accuracy on Test Set: {accuracy:.4f}')

# Mostrar métricas adicionais na amostra menor do conjunto de teste
print(classification_report(labels_test[:len(all_preds)], all_preds))


Accuracy on Test Set: 0.9881
              precision    recall  f1-score   support

           0       0.95      0.95      0.95       527
           1       0.99      0.99      0.99      3842

    accuracy                           0.99      4369
   macro avg       0.97      0.97      0.97      4369
weighted avg       0.99      0.99      0.99      4369



Os resultados que você obteve são impressionantes! Uma acurácia média de 0.97 é muito alta, especialmente considerando que você usou apenas uma época para treinar o modelo. Aqui estão algumas conclusões que podemos tirar:

1. **Desempenho do Modelo**: O modelo BERT (neuralmind/bert-base-portuguese-cased) parece ter se saído muito bem na tarefa de classificação de texto (hate e not_hate). Isso indica que o modelo foi capaz de capturar efetivamente os padrões nos dados de treinamento e generalizar bem para os dados de teste.

2. **Configuração de Treinamento**: A configuração de treinamento que você usou (tamanho do lote = 4, taxa de aprendizado = 2e-5, otimizador = Adam, função de perda = torch.nn.BCEWithLogitsLoss()) parece ser adequada para essa tarefa específica. O fato de que você conseguiu uma alta acurácia com apenas uma época sugere que o modelo convergiu rapidamente para uma solução ótima.

3. **Pré-processamento de Dados**: O pré-processamento de dados (padding com max_length = 128) foi eficaz para lidar com a variabilidade no comprimento dos tweets.

No entanto, é importante notar que, embora a acurácia seja alta, ainda é crucial avaliar o modelo em outras métricas (como precisão, recall, F1-score) para ter uma compreensão mais completa do desempenho do modelo. Além disso, seria útil realizar uma análise de erros para entender os tipos de erros que o modelo está cometendo. Isso pode fornecer insights sobre como melhorar ainda mais o modelo ou o processo de pré-processamento de dados.

Por fim, embora o modelo tenha apresentado um bom desempenho com apenas uma época, pode ser interessante experimentar com mais épocas para ver se o desempenho melhora ainda mais. No entanto, deve-se ter cuidado para evitar o sobreajuste.


Até a última atualização do meu conhecimento em janeiro de 2023, BERTimbau é um modelo de linguagem pré-treinado desenvolvido para o idioma português. O nome "BERTimbau" é uma combinação do acrônimo BERT (Bidirectional Encoder Representations from Transformers) e "tamborim", um instrumento musical brasileiro.

**Principais Características de BERTimbau:**
1. **Pré-treinamento:** BERTimbau foi treinado em uma grande quantidade de texto em português, utilizando tarefas como máscara de linguagem (MLM) e previsão da próxima sentença (NSP).

2. **Base do Modelo:** O modelo é construído com base na arquitetura de transformadores, que se mostrou eficaz para capturar relações contextuais em dados sequenciais.

3. **Tokenização:** Assim como outros modelos BERT, BERTimbau utiliza a tokenização WordPiece, que divide palavras em subpalavras.

**Aplicações Potenciais de BERTimbau:**
1. **Classificação de Texto:** Pode ser utilizado para tarefas de classificação binária (por exemplo, análise de sentimento) ou multiclasse (por exemplo, categorização de notícias).

2. **Análise de Sentimento:** Avaliar a polaridade (positiva, negativa, neutra) em textos em português.

3. **Named Entity Recognition (NER):** Identificar e classificar entidades nomeadas em textos, como nomes de pessoas, organizações, locais, etc.

4. **Tradução Automática:** BERTimbau pode ser usado para tarefas de tradução automática em português.

5. **Pergunta e Resposta (Q&A):** Aplicado em sistemas de pergunta e resposta, onde o modelo pode extrair informações relevantes de um contexto para responder perguntas.

6. **Sumarização de Texto:** Pode ser utilizado para resumir documentos ou artigos em português.

Lembre-se de que o desempenho específico do modelo em uma tarefa depende de diversos fatores, incluindo a qualidade dos dados de treinamento, o ajuste fino para a tarefa específica e outros parâmetros do modelo. Se você estiver considerando usar BERTimbau em um projeto, recomendo verificar a documentação mais recente e as práticas recomendadas fornecidas pelos desenvolvedores.